In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import xarray as xr

In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(embed_size, embed_size)
        self.keys = nn.Linear(embed_size, embed_size)
        self.queries = nn.Linear(embed_size, embed_size)
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0] #batch size

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]#sentence length so number of features ?

        values = self.values(values)  # (N, value_len, embed_size)
        keys = self.keys(keys)  # (N, key_len, embed_size)
        queries = self.queries(query)  # (N, query_len, embed_size)

        
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out


In [4]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


In [5]:
class EmbConv(nn.Module):
    def __init__(self,embed_size):
        super(EmbConv,self).__init__()
        self.conv1=nn.Conv2d(4,embed_size,kernel_size=2,stride=1,padding=1,dilation=2) ##problème kernel_size=2 ça dépasse si pas dilatation=2
 
    def forward(self,x):
        x=self.conv1(x)
        return x 
##Maintenant on a [batch_size,embed_size,h,w]


In [58]:
class PersonalEncoder(nn.Module):
    def __init__(self,full_data_size,embed_size,num_layer,heads,forward_expansion,dropout,max_length):
        super(PersonalEncoder,self).__init__()
        self.embed_size=embed_size
        self.input_embedding=EmbConv(embed_size)
        self.position_embedding=nn.Embedding(max_length,embed_size)
        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout=nn.Dropout(dropout)

    def forward(self,x):
        #x has size : [N,h,w,nb_features]
        #Position encoding :
        N,h,w,nb_features = x.shape
        position = np.arange(1, h*w + 1).reshape(h, w)
        position = np.tile(position, (N, 1, 1))
        position = torch.from_numpy(position)

        print('position size', position.size())
        ##on doit rajouter le nbre_features : 
        #position = position.repeat(embed_size, 1,1,1)
        #position = torch.permute(position,(1,2,3,0))

        
        
        #Input embedding + position embedding : 
        #x needs to be size [N,channels,h,w] for CNN 
        x=x.numpy()
        x=np.transpose(x, (0, 3, 1, 2))
        x=torch.from_numpy(x)
        print('x',x.size())
        
        #On remet x dans les dimensions initiales pour match the size of input_embedding : 
        x=self.input_embedding(x)
        #print("Input_embedding has been done and x is size :",x.size())
        x=x.detach().numpy()
        x=np.transpose(x, (0, 2, 3, 1))
        x=torch.from_numpy(x)
        #print(self.position_embedding(position).size())
        ##position est de taille    

        print(self.position_embedding.weight.size())
        
        out=self.dropout(x+self.position_embedding(position))
        print(out.size()) #[N,h,w,embed_size]

        ##Transformer block :
        ##out must be size : [N,value_len,embed_size]
        out = out.reshape(N, h * w, embed_size)
        for layer in self.layers:
            out = layer(out, out, out, mask=None)
            #print("One more layer done and the size of the output is :", out.size())

        return out

In [59]:
input = torch.randint(0,20,(2,20,10,4)) #N=2,h=20,w=10,channels=4

In [60]:
full_data_size=1000
embed_size=16
num_layers=2
forward_expansion=4
heads=8
dropout=0
max_length=2000
model=PersonalEncoder(full_data_size,embed_size,num_layers,heads,forward_expansion,dropout,max_length)

In [61]:
input = input.type(torch.float)
#model(input)
#input.size()

In [62]:
input_float = torch.randn((2,20,10,4)) #N=2,h=20,w=10,channels=4
input_float.size()

torch.Size([2, 20, 10, 4])

In [63]:
model(input_float).size()

position size torch.Size([2, 20, 10])
x torch.Size([2, 4, 20, 10])
torch.Size([2000, 16])
torch.Size([2, 20, 10, 16])


torch.Size([2, 200, 16])

In [64]:
intermediate=model(input_float)

position size torch.Size([2, 20, 10])
x torch.Size([2, 4, 20, 10])
torch.Size([2000, 16])
torch.Size([2, 20, 10, 16])


In [65]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)
        self.batchnorm1 = nn.BatchNorm1d(hidden_size)
        self.batchnorm2 = nn.BatchNorm1d(output_size)

    def forward(self, x):
        out = self.fc1(x)
        out=out.detach().numpy()
        out=np.transpose(out, (0, 2, 1))
        out=torch.from_numpy(out) 
        out = self.batchnorm1(out)
        out = self.relu1(out)
        out = self.dropout(out)
        out=out.detach().numpy()
        out=np.transpose(out, (0, 2, 1))
        out=torch.from_numpy(out) 
        out = self.fc2(out)
        out=out.detach().numpy()
        out=np.transpose(out, (0, 2, 1))
        out=torch.from_numpy(out) 
        out = self.batchnorm2(out)
        out = self.sigmoid(out)
        return out
    

In [66]:
MLP_model = MLP(input_size=embed_size, hidden_size=100, output_size=1)

In [98]:
class TransfoDownscaling(nn.Module):
    def __init__(self,full_data_size,embed_size,num_layers,heads,forward_expansion,dropout,max_length,input_size, hidden_size, output_size,):
        super(TransfoDownscaling, self).__init__()
        self.Encode = PersonalEncoder(full_data_size,embed_size,num_layers,heads,forward_expansion,dropout,max_length)
        self.MLP = MLP(input_size, hidden_size, output_size)

    def forward(self,x):
        print('taille de x en entrée :',x.size())
        x = self.Encode(x)
        print("taille de x en sortie de l'encodeur :",x.size())
        x = self.MLP(x)
        print('taille de x en sortie :',x.size())
        return x

        
full_data_size=5000
embed_size=16
num_layers=2
forward_expansion=4
heads=8
dropout=0
max_length=5000
model = TransfoDownscaling(full_data_size,embed_size,num_layers,heads,forward_expansion,dropout,max_length,input_size=embed_size, hidden_size=100, output_size=1)

print(input_float.size())
out= model(input_float)
#print(out.size())

torch.Size([2, 20, 10, 4])
taille de x en entrée : torch.Size([2, 20, 10, 4])
position size torch.Size([2, 20, 10])
x torch.Size([2, 4, 20, 10])
torch.Size([5000, 16])
torch.Size([2, 20, 10, 16])
taille de x en sortie de l'encodeur : torch.Size([2, 200, 16])
taille de x en sortie : torch.Size([2, 1, 200])


In [100]:
##TRAINING :    

In [101]:
#size of input = torch.randint(0,20,(2,20,10,4)) #N=2,h=20,w=10,channels=4 


In [102]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/data"
os.chdir(path)
var_janv_fev=pd.read_csv('janvier_fevrier_variables_leman.csv')
var_mars_avril=pd.read_csv('mars_avril_variables_leman.csv')

In [92]:
selected_cols = var_janv_fev[['rlon','rlat','temperature','humidity']]
with_temp_jf = torch.tensor(selected_cols.values)
selected_cols = var_mars_avril[['rlon','rlat','temperature','humidity']]
with_temp_ma = torch.tensor(selected_cols.values)

jf_input = with_temp_jf.reshape(60,60,4)
ma_input = with_temp_ma.reshape(60,60,4)
combined_input = torch.cat([jf_input.unsqueeze(0), ma_input.unsqueeze(0)], dim=0)
combined_input.size()

torch.Size([2, 60, 60, 4])

In [93]:
gev_params_janv_fev=pd.read_csv('gev_data_janv_fev.csv')
gev_params_mars_avril=pd.read_csv('gev_data_mars_avril.csv')

In [82]:
path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/mberthier/repos/Downscaling_CM/utils"
os.chdir(path)
import dataset_for_R 
out_janv_fev = dataset_for_R.get_precipitation_CNN_size('janvier','fevrier',lon_bnd=(-2.6, -1.42),lat_bnd=(-0.6, 0.58))
out_mars_avril=dataset_for_R.get_precipitation_CNN_size('mars','avril',lon_bnd=(-2.6, -1.42),lat_bnd=(-0.6, 0.58))

out_janv_fev.size()

torch.Size([60, 60, 22])

In [103]:
model.double()
model(combined_input).size()

taille de x en entrée : torch.Size([2, 60, 60, 4])
position size torch.Size([2, 60, 60])
x torch.Size([2, 4, 60, 60])
torch.Size([5000, 16])
torch.Size([2, 60, 60, 16])
taille de x en sortie de l'encodeur : torch.Size([2, 3600, 16])
taille de x en sortie : torch.Size([2, 1, 3600])


torch.Size([2, 1, 3600])

In [94]:
class CustomCRPSLoss(nn.Module):
    def __init__(self):
        super(CustomCRPSLoss, self).__init__()

    def forward(self, y_pred, y_true):
        term_one = torch.mean(torch.abs(y_pred - y_true), dim=-1)
        term_two = torch.mean(torch.abs(
        torch.unsqueeze(y_pred, -1) - torch.unsqueeze(y_pred, -2)), dim=(-2, -1))
        half = torch.tensor(-0.5, dtype=term_two.dtype)
        loss = term_one + half * term_two
        loss = torch.mean(loss)
        return loss

In [96]:
#x has size : [N,h,w,nb_features] while for CNN : [batch,channels,h,w]
num_epoch = 5
model=model.double()
optimizer= torch.optim.Adam(model.parameters(), lr=0.1)
loss_crps=CustomCRPSLoss()
loss_list=[]
input = combined_input
nb_lon=input.shape[2]
nb_lat=input.shape[3]

for epoch in range(num_epoch):

    if epoch%2==0 :
        output=model(input)
        batch_output = out_janv_fev
        loc = output[0,0,:,:]
        size = (output.shape[2],output.shape[3],batch_output.shape[2])

        u = torch.rand(size)
        #print(size)
        loc = loc.unsqueeze(-1).repeat(1, 1, size[2])
        loc = torch.transpose(loc, 0, 1)
        #print(loc.size())

        ##On fixe xi et scale
        
        c = torch.tensor(gev_params_janv_fev['shape'])
    
        c = c.reshape(60, 60)
        c = torch.transpose(c, 0,1) ##oujours 60,60
        c = c.repeat(22,1,1)
        c = torch.permute(c,(1,2,0))
   
        
        scale = torch.tensor(gev_params_janv_fev['scale'])
        scale = scale.reshape(60, 60)
        scale = torch.transpose(scale, 0, 1)  # Toujours 60, 60
        scale = scale.repeat(22, 1, 1)
        scale = torch.permute(scale, (1, 2, 0))
                
        sample = loc + (torch.pow(-torch.log(u), -c) - 1) * scale / c
        #print('sample',sample)
       
        loss1=loss_crps(sample,batch_output)
        loss_list.append(torch.detach(loss1).numpy())
        optimizer.zero_grad()
        loss1.backward()
        optimizer.step()
    else : 
        output=model(input)
        batch_output = out_mars_avril
        loc = output[0,0,:,:]
        size = (output.shape[2],output.shape[3],batch_output.shape[2])

        u = torch.rand(size)

        loc = loc.unsqueeze(-1).repeat(1, 1, size[2])
        loc = torch.transpose(loc, 0, 1)
        #print(loc.size())

        ##On fixe xi et scale
        
        c = torch.tensor(gev_params_mars_avril['shape'])
    
        c = c.reshape(60, 60)
        c = torch.transpose(c, 0,1) ##oujours 60,60
        c = c.repeat(22,1,1)
        c = torch.permute(c,(1,2,0))
   
        
        scale = torch.tensor(gev_params_mars_avril['scale'])
        scale = scale.reshape(60, 60)
        scale = torch.transpose(scale, 0, 1)  # Toujours 60, 60
        scale = scale.repeat(22, 1, 1)
        scale = torch.permute(scale, (1, 2, 0))
        
        
        sample = loc + (torch.pow(-torch.log(u), -c) - 1) * scale / c
        #print('sample',sample)
       
        loss1=loss_crps(sample,batch_output)
        loss_list.append(torch.detach(loss1).numpy())
        optimizer.zero_grad()
        loss1.backward()
        optimizer.step()
   
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epoch}], Loss: {loss1.item():.4f}')

plt.plot(loss_list[3:])
plt.show()

taille de x en entrée : torch.Size([2, 60, 60, 4])
position size torch.Size([2, 60, 60])
x torch.Size([2, 4, 60, 60])
torch.Size([5000, 16])
torch.Size([2, 60, 60, 16])
taille de x en sortie : torch.Size([2, 1, 3600])


IndexError: too many indices for tensor of dimension 3